In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
test_data = pd.read_csv('Liver test.csv')

In [3]:
train_data = pd.read_csv('Liver train.csv')

In [4]:
# converting the age (in days) into years
from datetime import date, timedelta

# Define the age in days
age_in_days = train_data['Age']
# Average number of days in a year (considering leap years)
average_days_in_year = 365.25

# Calculate age in years as a float
train_data['Age(inYears)'] = train_data['Age'] / average_days_in_year
train_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,...,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,Age(inYears)
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,...,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D,58.951403
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,...,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C,52.668036
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,...,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D,37.582478
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,...,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C,50.540726
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,...,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C,45.607118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,7900,1166,D-penicillamine,16839,F,N,N,N,N,0.8,...,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,C,46.102669
7901,7901,1492,Placebo,17031,F,N,Y,N,N,0.9,...,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,C,46.628337
7902,7902,1576,D-penicillamine,25873,F,N,N,Y,S,2.0,...,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,D,70.836413
7903,7903,3584,D-penicillamine,22960,M,N,Y,N,N,0.7,...,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,D,62.861054


In [5]:
cat_cols=[]
unique_values = {}
for column in train_data.select_dtypes(include=['object']):
    cat_cols.append(column)
    unique_values[column] = train_data[column].unique()
    
unique_values

{'Drug': array(['D-penicillamine', 'Placebo'], dtype=object),
 'Sex': array(['M', 'F'], dtype=object),
 'Ascites': array(['N', 'Y'], dtype=object),
 'Hepatomegaly': array(['N', 'Y'], dtype=object),
 'Spiders': array(['N', 'Y'], dtype=object),
 'Edema': array(['N', 'Y', 'S'], dtype=object),
 'Status': array(['D', 'C', 'CL'], dtype=object)}

In [6]:
cat_cols

['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Status']

In [7]:
# Drug Sex Ascites Hepatomegaly  Spiders 
# Edema
def process_train(df):
    df["Drug"] = df["Drug"].map({"Placebo": 0, "D-penicillamine": 1})
    df["Sex"] = df["Sex"].map({"M": 0, "F": 1})
    df['Ascites'] = df['Ascites'].map({"N":0, "Y":1})
    df['Hepatomegaly'] = df['Hepatomegaly'].map({"N":0, "Y":1})
    df['Spiders'] = df['Spiders'].map({"N":0, "Y":1})
    df['Edema'] = df['Edema'].map({"N":0, "Y":1, "S":2})
    
    df['Status'] = df['Status'].map({"C":0, "CL":1, "D":2})
    return df

train= process_train(train_data)
train

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,...,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,Age(inYears)
0,0,999,1,21532,0,0,0,0,0,2.3,...,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,2,58.951403
1,1,2574,0,19237,1,0,0,0,0,0.9,...,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,0,52.668036
2,2,3428,0,13727,1,0,1,1,1,3.3,...,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,2,37.582478
3,3,2576,0,18460,1,0,0,0,0,0.6,...,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,0,50.540726
4,4,788,0,16658,1,0,1,0,0,1.1,...,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,0,45.607118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,7900,1166,1,16839,1,0,0,0,0,0.8,...,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,0,46.102669
7901,7901,1492,0,17031,1,0,1,0,0,0.9,...,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,0,46.628337
7902,7902,1576,1,25873,1,0,0,1,2,2.0,...,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,2,70.836413
7903,7903,3584,1,22960,0,0,1,0,0,0.7,...,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,2,62.861054


In [8]:
##def plot_heatmap(df, exclude_columns=[]):
    
#    df_filtered = df.drop(columns=exclude_columns)
   
#    correlation_matrix = df_filtered.corr()
   
#    plt.figure(figsize=(10, 8))
   
#    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
   
 #   plt.show()



#plot_heatmap(train, exclude_columns=cat_cols)

In [9]:
test_data = pd.read_csv('Liver test.csv')
test_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7905,3839,D-penicillamine,19724,F,N,Y,N,N,1.2,546.0,3.37,65.0,1636.0,151.90,90.0,430.0,10.6,2.0
1,7906,2468,D-penicillamine,14975,F,N,N,N,N,1.1,660.0,4.22,94.0,1257.0,151.90,155.0,227.0,10.0,2.0
2,7907,51,Placebo,13149,F,N,Y,N,Y,2.0,151.0,2.96,46.0,961.0,69.75,101.0,213.0,13.0,4.0
3,7908,2330,D-penicillamine,20510,F,N,N,N,N,0.6,293.0,3.85,40.0,554.0,125.55,56.0,270.0,10.6,2.0
4,7909,1615,D-penicillamine,21904,F,N,Y,N,N,1.4,277.0,2.97,121.0,1110.0,125.00,126.0,221.0,9.8,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13171,2870,Placebo,12279,F,N,N,N,N,1.3,302.0,3.43,75.0,1345.0,145.00,44.0,181.0,10.6,3.0
5267,13172,1770,Placebo,24803,F,N,N,N,N,0.5,219.0,4.09,121.0,663.0,79.05,94.0,311.0,9.7,3.0
5268,13173,3707,D-penicillamine,16990,F,N,Y,N,N,0.8,315.0,4.09,13.0,1637.0,170.50,70.0,426.0,10.9,3.0
5269,13174,1216,Placebo,11773,F,N,N,N,N,0.7,329.0,3.80,52.0,678.0,57.00,126.0,306.0,10.2,1.0


# XG Boost Model

In [22]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [23]:
X = train_data.drop('Status', axis=1)  # Features
y = train_data['Status']  # Target variable

In [24]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# Create and train XGBoost model
xgb_model = xgb.XGBClassifier()  # You can adjust parameters here
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [26]:
# Make predictions on testing data
y_pred = xgb_model.predict(X_test)

In [27]:
# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8311
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       966
           1       0.56      0.17      0.26        52
           2       0.82      0.74      0.78       563

    accuracy                           0.83      1581
   macro avg       0.74      0.61      0.64      1581
weighted avg       0.82      0.83      0.82      1581

